### Importación de librerías y archivos
En las siguientes líneas de código se importaran los materiales de trabajo necesarios para desarrollar el caso de uso, en esto se incluyen las librerías y los datos

In [1]:
# Importar las librerías necesarias según el análisis que se vaya a realizar
# Librería para comando de sistema
import os
# Librería para manejo de datos
import pandas as pd
# Librería para manejo de fechas
from datetime import date

In [3]:
# cargar los datos en csv
data= pd.read_csv('loan_data_2007_2014.csv', sep=',',encoding='latin-1')
# Visualizar los datos
data.head()

C:\Users\Edwar\AppData\Local\Temp\ipykernel_2540\111207929.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv('loan_data_2007_2014.csv', sep=',',encoding='latin-1')


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000,3000,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Identificacion del tipo de datos


In [4]:
# Revisar los tipos de datos del dataset
data.dtypes

id                    int64
member_id             int64
loan_amnt             int64
funded_amnt           int64
funded_amnt_inv     float64
                     ...   
all_util            float64
total_rev_hi_lim    float64
inq_fi              float64
total_cu_tl         float64
inq_last_12m        float64
Length: 74, dtype: object

In [5]:
#porcentaje de nulos por variable
data.isna().sum()/len(data)

id                  0.000000
member_id           0.000000
loan_amnt           0.000000
funded_amnt         0.000000
funded_amnt_inv     0.000000
                      ...   
all_util            1.000000
total_rev_hi_lim    0.150715
inq_fi              1.000000
total_cu_tl         1.000000
inq_last_12m        1.000000
Length: 74, dtype: float64

In [8]:
data["grade"].value_counts()

B    136929
C    125293
D     76888
A     74867
E     35757
F     13229
G      3322
Name: grade, dtype: int64

In [16]:
data2=data[["grade","id"]]


In [17]:
data2.dtypes

grade    object
id        int64
dtype: object

In [5]:
# Obtener estadísticas básica
data['codigo_postal_vendedor'].describe()

count    10134.000000
mean     49991.684626
std      22463.895340
min       5001.000000
25%      50001.000000
50%      52323.000000
75%      54003.000000
max      99001.000000
Name: codigo_postal_vendedor, dtype: float64

# ¡Cuidado con este error de interpretación!

Al ser un dato de identificación es un dato categórico y no tiene ninguna interpretación describir las estadísticas descriptivas básicas

In [6]:
# lo mejor es convertirlo en categórico
data['codigo_postal_vendedor']=data['codigo_postal_vendedor'].astype(str)
data.dtypes

orden_id                          object
order_item_id                     object
producto_id                       object
vendedor_id                       object
fecha_envio_limite                object
precio                           float64
valor_flete                      float64
codigo_postal_vendedor            object
ciudad_vendedor                   object
departamento_vendedor             object
nombre_categoria_producto         object
longitud_nombre_producto         float64
longitud_descripcion_producto    float64
cantidad_fotos_producto          float64
peso_g_producto                  float64
longitud_cm_producto             float64
altura_cm_producto               float64
ancho_cm_producto                float64
dtype: object

### 4. Datos nulos

Los datos nulos son los registros que por cualquier razón no registran ningún dato, que por lo tanto van a generar un ruido o un sesgo (dependiendo del análisis) y que va a alterar el manejo de los datos para realizar análisis de alto nivel

**Contexto:** No existen datos faltantes en la base, ya que si no están los datos de las órdenes no es posible encontrar cómo realizar el envío

In [7]:
## Función para revisar nulos
# Muestra variable por variable qué porcentaje de nulos se tienen
data.isna().sum()/len(data)

orden_id                         0.00000
order_item_id                    0.00000
producto_id                      0.00000
vendedor_id                      0.00000
fecha_envio_limite               0.00000
precio                           0.00000
valor_flete                      0.00000
codigo_postal_vendedor           0.00000
ciudad_vendedor                  0.00000
departamento_vendedor            0.00000
nombre_categoria_producto        0.00227
longitud_nombre_producto         0.00227
longitud_descripcion_producto    0.00227
cantidad_fotos_producto          0.00227
peso_g_producto                  0.00227
longitud_cm_producto             0.00227
altura_cm_producto               0.00227
ancho_cm_producto                0.00227
dtype: float64

In [8]:
data.shape

(10134, 18)

Las variables involucradas apenas tienen data faltante menor al 1%, por lo tanto se debe decidir cuál criterio utilizar para la data faltante

1. Métodos de eliminacion

Los métodos de eliminación solo funcionan para determinados conjuntos de datos en los que a los participantes les faltan campos. Hay varios métodos de eliminación: dos comunes incluyen la eliminación por lista y la eliminación por pares. Significa eliminar cualquier participante o entrada de datos con valores perdidos. La eliminación por pares es el proceso de eliminar información cuando falta un punto de datos en particular, vital para las pruebas. La eliminación por pares guarda más datos en comparación con la eliminación similar porque la primera solo elimina las entradas donde las variables eran necesarias para la prueba, mientras que la última elimina las entradas completas si falta algún dato, independientemente de su importancia.

2. Análisis de regresión

La regresión es útil para manejar datos faltantes porque puede usarse para predecir el valor nulo usando otra información del conjunto de datos. Los métodos de regresión pueden tener éxito para encontrar los datos faltantes, pero esto depende en gran medida de qué tan bien conectados estén los datos restantes. 

3. Técnicas de imputación de datos

La imputación promedio utiliza el valor promedio de las respuestas de otras entradas de datos para completar los valores faltantes. La imputación de punto comúnes cuando los datos utilizan el punto medio o el valor elegido con más frecuencia. 

4. Mantener los vacíos

No podemos evitar situaciones como estas por completo porque hay varios pasos correctivos que los científicos de datos deben tomar para asegurarse de que no afecten negativamente el proceso de análisis. 

**Análisis:** En este caso al no contar con el nombre, categoría ni medidas del producto es necesario eliminar estos registros, así como la orden creada y hablar con los clientes para generar la devolución de su dinero

In [9]:
# eliminar los datos vacío
data=data.dropna()
data.shape

(10111, 18)

### 4. Duplicados

Identificar los datos (registros) que por alguna razón tienen columnas repetidas en todas las variables, es decir está generando data erronea ya que está mostrando datos que ya están en la base

**Contexto:** Existen órdenes duplicadas para realizar los envíos

In [10]:
#Revisar cuáles registros están duplicados
data.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
10129    False
10130    False
10131    False
10132    False
10133    False
Length: 10111, dtype: bool

In [11]:
# establecer un método para eliminar duplicados
data=data.drop_duplicates()
data.head(5)

,orden_id,order_item_id,producto_id,vendedor_id,fecha_envio_limite,precio,valor_flete,codigo_postal_vendedor,ciudad_vendedor,departamento_vendedor,nombre_categoria_producto,longitud_nombre_producto,longitud_descripcion_producto,cantidad_fotos_producto,peso_g_producto,longitud_cm_producto,altura_cm_producto,ancho_cm_producto
0,107500PO59A,A,PO59,VE5389,7/04/2018 18:12,271.86,30.72,52435,Mallama,Nariño,Productos ecoamigables,6.0,7.0,27.0,2486.0,17.0,11.0,14.0
1,37493PS22B,B,PS22,VE1558,20/10/2017 9:07,115.73,4.68,52203,Colon,Nariño,Carnicería,10.0,31.0,20.0,256.0,43.0,2.0,21.0
2,28050PK20B,B,PK20,VE9159,17/08/2017 8:15,432.99,82.70,66001,Pereira,Risaralda,Deportes,25.0,5.0,4.0,5270.0,9.0,27.0,29.0
3,52187PA10A,A,PA10,VE3159,23/09/2017 23:27,108.38,35.39,52435,Mallama,Nariño,Electrodomésticos,10.0,1.0,6.0,734.0,46.0,48.0,22.0
4,84639PR12A,A,PR12,VE5090,7/01/2018 11:50,51.50,11.10,73001,Ibague,Tolima,Frutas y verduras,23.0,16.0,35.0,884.0,45.0,26.0,18.0


In [12]:
#Mostrar la totalidad de datos duplicados, la función .sum() muestra la suma de registros
data.duplicated().sum()

0

**Análisis:** No existen órdenes duplicadas

### 5. Elegir el formato
Cuando se trabaja con datos es posible encontrarse con diferentes tipos de datos y es muy importante encontrar el tipo de dato correcto, es decir, saber si se trabaja con reales, con enteros, categorías, fechas, etc...

**Hipótesis:** Es posible realizar el cálculo de la fecha de envío hasta hoy

In [13]:
## Arreglo de datos
# Se trata de restar dos fechas
hoy = pd.to_datetime("now")
hoy


C:\Users\Edwar\anaconda3\lib\site-packages\pandas\core\arrays\datetimes.py:2199: FutureWarning: The parsing of 'now' in pd.to_datetime without `utc=True` is deprecated. In a future version, this will match Timestamp('now') and Timestamp.now()
  result, tz_parsed = tslib.array_to_datetime(


Timestamp('2022-07-15 04:06:08.665978')

In [14]:
#restar los días que hay entre hoy y la fecha de pedido
data['fecha_envio_limite']

0         7/04/2018 18:12
1         20/10/2017 9:07
2         17/08/2017 8:15
3        23/09/2017 23:27
4         7/01/2018 11:50
               ...       
10129     1/04/2018 11:38
10130    21/10/2017 17:09
10131     10/10/2017 8:37
10132     13/11/2017 3:20
10133    26/12/2017 19:34
Name: fecha_envio_limite, Length: 10111, dtype: object

In [15]:
data['fecha_envio_limite']-hoy

TypeError: unsupported operand type(s) for -: 'numpy.ndarray' and 'Timestamp'

# ¡Cuidado con el error, hay que revisar qué formato se está utlizando!

In [16]:
# Revisar el tipo de dato que es la columnda de fechas
data[['fecha_envio_limite']].dtypes

fecha_envio_limite    object
dtype: object

In [17]:
## Cambiar el tipo de dato
# El tipo de dato de fecha para el lenguaje de programación es un texto, por lo tanto,
# no se pueden realizar operaciones de ninguna, es necesario cambiar el tipo de dato
data['fecha_envio_limite']=pd.to_datetime(data['fecha_envio_limite'])

In [18]:
# revisar que el tipo de dato es de fecha
data[['fecha_envio_limite']].dtypes

fecha_envio_limite    datetime64[ns]
dtype: object

In [19]:
# Hacer operaciones con el tipo de dato correcto
# como ya la columna es el tipo de dato correcto se puede hacer la resta para obtener los días
hoy-data['fecha_envio_limite']

0       1471 days 09:54:08.665978
1       1728 days 18:59:08.665978
2       1792 days 19:51:08.665978
3       1755 days 04:39:08.665978
4       1474 days 16:16:08.665978
                   ...           
10129   1652 days 16:28:08.665978
10130   1727 days 10:57:08.665978
10131   1738 days 19:29:08.665978
10132   1705 days 00:46:08.665978
10133   1661 days 08:32:08.665978
Name: fecha_envio_limite, Length: 10111, dtype: timedelta64[ns]

**Análisis:** Es posible realizar el cálculo de la fecha hasta hoy, sin embargo, es muy importante tener en cuenta el tipo de dato que se está operando

![Banner-Introducci-n.png](https://i.postimg.cc/VkCcqsvv/Banner-Introducci-n.png)